### LR test on all 29 features, bi-weekly

In [ ]:
# Another test, move the ground truth one day after, use the data from previous day to predict later day
import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import time
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = fileList[-1000:]

selectedList = random.sample(fileList, 2000)
restList = list(set(fileList) - set(selectedList))

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, 29])
y = np.zeros([1, 1])

for f in selectedList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    y1 = data['w_up_down'].values
    X1 = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data
    X1.fillna(0, inplace=True)
    X1 = X1.replace(np.inf, 0)  # Replace inf with 0
    
#     try:
    X1 = scaler.fit_transform(X1)
    y0 = np.zeros([1,1])
    y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

    X = np.vstack((X, X1))
    y = np.append(y, y1)  # stack all labels

    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
#     except:
#         print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
# Prepare a test set of data

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = fileList[:2000]

# fileList = random.sample(fileList, 100)

fileList = random.sample(restList, 500)

# fileList = ['002169.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, 29])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    X1_test = X1_test.replace(np.inf, 0)  # Replace inf with 0
    try:
        X1_test = scaler.fit_transform(X1_test)
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

### Modeling - Logestic Regression

In [ ]:
# Build classifier using logistic regression
# Easy to interpret
# Linear models tend to perform well on sparse datasets like this one
# Learn very fast compared to other algorithms.

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X, y)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, lr.predict(X_test))))

# Hyperparameter C adjusts the regularization. We found C = 0.05 performs the best

In [ ]:
# newtarget = [0 if i < 500 else 1 for i in range(1000)]
# final_model = LogisticRegression(C=0.5)
# final_model.fit(X, target)
# model = final_model.fit(X_train, y_train)
# print ("Final Accuracy: %s" 
#        % accuracy_score(newtarget, final_model.predict(X_test)))

### Confusion Matrix - Logestic Regression

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
# Precision, recall, f score

from sklearn.metrics import precision_recall_fscore_support

score_spt_macro = precision_recall_fscore_support(y_test, y_pred, average='macro')
score_spt_micro = precision_recall_fscore_support(y_test, y_pred, average='micro')
score_spt_weighted = precision_recall_fscore_support(y_test, y_pred, average='weighted')
score_spt_none = precision_recall_fscore_support(y_test, y_pred, average=None)

print('Score support of precision recall and f1 score: macro: {}, micro: {}, weighted: {}, none: {}'.format(score_spt_macro, score_spt_micro, score_spt_weighted, score_spt_none))
print('Macro precision is: {}, micro precision is: {}, weighted precision is: {}'.format(score_spt_macro[0], score_spt_micro[0], score_spt_weighted[0]))
print('Macro recall is: {}, micro recall is: {}, weighted recall is: {}'.format(score_spt_macro[1], score_spt_micro[1], score_spt_weighted[1]))
print('Macro f1 score is: {}, micro f1 score is: {}, weighted f1 score is: {}'.format(score_spt_macro[2], score_spt_micro[2], score_spt_weighted[2]))
print('Score support of precision recall and f1 score: None: {}'.format(score_spt_none))

### Model - SVM

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC

# hyper parameter test

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lsvm = LinearSVC(C=c)
    lsvm.fit(X, y)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, lsvm.predict(X_test))))

### Confusion Matrix - SVM

In [ ]:
y_pred = lsvm.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
# Precision, recall, f score

from sklearn.metrics import precision_recall_fscore_support

score_spt_macro = precision_recall_fscore_support(y_test, y_pred, average='macro')
score_spt_micro = precision_recall_fscore_support(y_test, y_pred, average='micro')
score_spt_weighted = precision_recall_fscore_support(y_test, y_pred, average='weighted')
score_spt_none = precision_recall_fscore_support(y_test, y_pred, average=None)

print('Score support of precision recall and f1 score: macro: {}, micro: {}, weighted: {}, none: {}'.format(score_spt_macro, score_spt_micro, score_spt_weighted, score_spt_none))
print('Macro precision is: {}, micro precision is: {}, weighted precision is: {}'.format(score_spt_macro[0], score_spt_micro[0], score_spt_weighted[0]))
print('Macro recall is: {}, micro recall is: {}, weighted recall is: {}'.format(score_spt_macro[1], score_spt_micro[1], score_spt_weighted[1]))
print('Macro f1 score is: {}, micro f1 score is: {}, weighted f1 score is: {}'.format(score_spt_macro[2], score_spt_micro[2], score_spt_weighted[2]))
print('Score support of precision recall and f1 score: None: {}'.format(score_spt_none))

### Model - Naive Bayes

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import BernoulliNB

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    bnbc = BernoulliNB(binarize=None, alpha=c)
    bnbc.fit(X, y)
    
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, bnbc.predict(X_test))))

### Confusion Matrix - NB

In [ ]:
y_pred = bnbc.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
# Precision, recall, f score

from sklearn.metrics import precision_recall_fscore_support

score_spt_macro = precision_recall_fscore_support(y_test, y_pred, average='macro')
score_spt_micro = precision_recall_fscore_support(y_test, y_pred, average='micro')
score_spt_weighted = precision_recall_fscore_support(y_test, y_pred, average='weighted')
score_spt_none = precision_recall_fscore_support(y_test, y_pred, average=None)

print('Score support of precision recall and f1 score: macro: {}, micro: {}, weighted: {}, none: {}'.format(score_spt_macro, score_spt_micro, score_spt_weighted, score_spt_none))
print('Macro precision is: {}, micro precision is: {}, weighted precision is: {}'.format(score_spt_macro[0], score_spt_micro[0], score_spt_weighted[0]))
print('Macro recall is: {}, micro recall is: {}, weighted recall is: {}'.format(score_spt_macro[1], score_spt_micro[1], score_spt_weighted[1]))
print('Macro f1 score is: {}, micro f1 score is: {}, weighted f1 score is: {}'.format(score_spt_macro[2], score_spt_micro[2], score_spt_weighted[2]))
print('Score support of precision recall and f1 score: None: {}'.format(score_spt_none))

### Model a simple MLP NN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define nn

num_pc = 29

# count the time, start point
start = time.time()
    
# create model
model = Sequential()
model.add(Dense(6, input_dim=num_pc, activation='relu'))
model.add(Dense(num_pc, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Compile model
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, batch_size=100)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# count the time, end point
end = time.time()

time_elapse = end - start
print(time_elapse)

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=10, verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
# Precision, recall, f score

from sklearn.metrics import precision_recall_fscore_support

score_spt_macro = precision_recall_fscore_support(y_test, y_pred, average='macro')
score_spt_micro = precision_recall_fscore_support(y_test, y_pred, average='micro')
score_spt_weighted = precision_recall_fscore_support(y_test, y_pred, average='weighted')
score_spt_none = precision_recall_fscore_support(y_test, y_pred, average=None)

print('Score support of precision recall and f1 score: macro: {}, micro: {}, weighted: {}, none: {}'.format(score_spt_macro, score_spt_micro, score_spt_weighted, score_spt_none))
print('Macro precision is: {}, micro precision is: {}, weighted precision is: {}'.format(score_spt_macro[0], score_spt_micro[0], score_spt_weighted[0]))
print('Macro recall is: {}, micro recall is: {}, weighted recall is: {}'.format(score_spt_macro[1], score_spt_micro[1], score_spt_weighted[1]))
print('Macro f1 score is: {}, micro f1 score is: {}, weighted f1 score is: {}'.format(score_spt_macro[2], score_spt_micro[2], score_spt_weighted[2]))
print('Score support of precision recall and f1 score: None: {}'.format(score_spt_none))

In [ ]:
X.shape

In [ ]:
# Keras CNN but not useful
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dropout

TIME_PERIODS = 1
num_pc = 29
num_classes = 1
input_shape = num_pc

# count the time, start point
start = time.time()

model_m = Sequential()
model_m.add(Reshape((TIME_PERIODS, num_pc), input_shape=(input_shape,)))
model_m.add(Conv1D(100, 10, activation='relu', input_shape=(TIME_PERIODS, num_pc)))
model_m.add(Conv1D(100, 10, activation='relu'))
model_m.add(MaxPooling1D(3))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(GlobalAveragePooling1D())
model_m.add(Dropout(0.5))
model_m.add(Dense(num_classes, activation='softmax'))
print(model_m.summary())

# define nn

# create model

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Compile model
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, batch_size=100)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# count the time, end point
end = time.time()

time_elapse = end - start
print(time_elapse)

### LSTM - 29 features

In [ ]:
import pandas as pd
from math import sqrt
from matplotlib import pyplot
# from sklearn import preprocessing
from numpy import array
from numpy import argmax
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


N_FEATURES = 29
N_TIME_STEPS = 10

# datasetpre1 = X  # delete 'index_col=0' in bracket because of lack of index
# datasetpre = pd.get_dummies(datasetpre1,drop_first=True)  # GET DUMMMIE and drop
# X = datasetpre.values
# # integer encode, IF OUT OF INDEX ERROR, CLOSE THE DISTRIBUTION AND RELAUNCH
# # Comment label encoder because of naming error, only onehotencoder works fine now
# label_encoder_0 = LabelEncoder()
# #X[:,0] = label_encoder_0.fit_transform(values[:,0])
# X[:,0] = label_encoder_0.fit_transform(datasetpre.values[:,0])
# label_encoder_1 = LabelEncoder()
# #X[:,1] = label_encoder_1.fit_transform(values[:,1])
# X[:,1] = label_encoder_1.fit_transform(datasetpre.values[:,1])
# label_encoder_2 = LabelEncoder()
# #X[:,2] = label_encoder_2.fit_transform(values[:,2])
# X[:,2] = label_encoder_2.fit_transform(datasetpre.values[:,2])
# # print(integer_encoded)

# # Onehotencoder will make the matrix really large, not fit second-based data structure
# #onehotencoder = OneHotEncoder(categorical_features = "all")
# #X = onehotencoder.fit_transform(X).toarray()
# # ensure all data is float
# X = X.astype('float32')
# # normalize features
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(X)
# frame as supervised learning
reframed = series_to_supervised(X, N_TIME_STEPS, 1)
print(reframed.shape)

# # binary encode
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)
# # invert first example
# inverted = label_encoder.inverse_transform([argmax(onehot_encoded[1, :])])
# print(inverted)

In [ ]:
y_new = y[:-10]

In [ ]:
# split into train and test sets
X = reframed.values
N_TRAIN_TIME = 591200
train = X[:N_TRAIN_TIME, :]
test = X[N_TRAIN_TIME:, :]
# split into input and outputs
N_OBS = N_TIME_STEPS * N_FEATURES
train_X = train[:, :N_OBS]
train_y = y_new[:N_TRAIN_TIME]
test_X = test[:, :N_OBS]
test_y = y_new[N_TRAIN_TIME:]
print(train_X.shape, len(train_X), train_y.shape)
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], N_TIME_STEPS, N_FEATURES))
test_X = test_X.reshape((test_X.shape[0], N_TIME_STEPS, N_FEATURES))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
from keras import metrics
from keras import backend as K

# Below is for F1 metrics
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
#         """Recall metric.

#         Only computes a batch-wise average of recall.

#         Computes the recall, a metric for multi-label classification of
#         how many relevant items are selected.
#         """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
#         """Precision metric.

#         Only computes a batch-wise average of precision.

#         Computes the precision, a metric for multi-label classification of
#         how many selected items are relevant.
#         """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

start = time.time()

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
#model.compile(loss='mae', optimizer='adam')
#model.compile(loss='mae', optimizer='adam', metrics=[metrics.mae, metrics.categorical_accuracy])
model.compile(loss='mae',
              optimizer= "adam",
              metrics=[f1, metrics.binary_accuracy, metrics.mean_squared_error, metrics.mean_absolute_error])
# fit network
#history = model.fit(train_X, train_y, epochs=6, batch_size=1440, validation_data=(test_X, test_y), verbose=2, shuffle=False)
history = model.fit(train_X, train_y, epochs=50, batch_size=3000, 
                    validation_data=(test_X, test_y), shuffle=True)
score = model.evaluate(test_X, test_y, batch_size=3000)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

end = time.time()

time_elapse = end - start
print(time_elapse)

In [ ]:
y_pred = model.predict_classes(test_X, batch_size=10, verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(test_y, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
# scores = model.evaluate(test_X, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("\n%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
print("\n%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
print("\n%s: %.2f%%" % (model.metrics_names[4], scores[4]*100))

In [ ]:
# Precision, recall, f score

from sklearn.metrics import precision_recall_fscore_support

score_spt_macro = precision_recall_fscore_support(test_y, y_pred, average='macro')
score_spt_micro = precision_recall_fscore_support(test_y, y_pred, average='micro')
score_spt_weighted = precision_recall_fscore_support(test_y, y_pred, average='weighted')
score_spt_none = precision_recall_fscore_support(test_y, y_pred, average=None)

print('Score support of precision recall and f1 score: macro: {}, micro: {}, weighted: {}, none: {}'.format(score_spt_macro, score_spt_micro, score_spt_weighted, score_spt_none))
print('Macro precision is: {}, micro precision is: {}, weighted precision is: {}'.format(score_spt_macro[0], score_spt_micro[0], score_spt_weighted[0]))
print('Macro recall is: {}, micro recall is: {}, weighted recall is: {}'.format(score_spt_macro[1], score_spt_micro[1], score_spt_weighted[1]))
print('Macro f1 score is: {}, micro f1 score is: {}, weighted f1 score is: {}'.format(score_spt_macro[2], score_spt_micro[2], score_spt_weighted[2]))
print('Score support of precision recall and f1 score: None: {}'.format(score_spt_none))

### LSTM - 5 Principal Component

In [ ]:
# Another test, move the ground truth one day after, use the data from previous day to predict later day
import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import time
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

num_pc = 5
processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = fileList[-1000:]

# fileList = random.sample(fileList, 500)

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

selectedList = random.sample(fileList, 2000)
restList = list(set(fileList) - set(selectedList))

scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, num_pc])
y = np.zeros([1, 1])

for f in selectedList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1 = data['w_up_down'].values
        X1 = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data
        X1.fillna(0, inplace=True)
        X1 = X1.replace(np.inf, 0)  # Replace inf with 0
    
        X1 = scaler.fit_transform(X1)

        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1)
        X1 = ipca.transform(X1)

        y0 = np.zeros([1,1])
        y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

        X = np.vstack((X, X1))
        y = np.append(y, y1)  # stack all labels

        X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y = y[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
# Prepare a test set of data

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = fileList[:2000]

fileList = random.sample(restList, 500)

# fileList = ['002169.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, num_pc])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    X1_test = X1_test.replace(np.inf, 0)  # Replace inf with 0
    try:
        X1_test = scaler.fit_transform(X1_test)
        
        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1_test)
        X1_test = ipca.transform(X1_test)
        
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
import pandas as pd
from math import sqrt
from matplotlib import pyplot
# from sklearn import preprocessing
from numpy import array
from numpy import argmax
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


N_FEATURES = 5
N_TIME_STEPS = 10

# datasetpre1 = X  # delete 'index_col=0' in bracket because of lack of index
# datasetpre = pd.get_dummies(datasetpre1,drop_first=True)  # GET DUMMMIE and drop
# X = datasetpre.values
# # integer encode, IF OUT OF INDEX ERROR, CLOSE THE DISTRIBUTION AND RELAUNCH
# # Comment label encoder because of naming error, only onehotencoder works fine now
# label_encoder_0 = LabelEncoder()
# #X[:,0] = label_encoder_0.fit_transform(values[:,0])
# X[:,0] = label_encoder_0.fit_transform(datasetpre.values[:,0])
# label_encoder_1 = LabelEncoder()
# #X[:,1] = label_encoder_1.fit_transform(values[:,1])
# X[:,1] = label_encoder_1.fit_transform(datasetpre.values[:,1])
# label_encoder_2 = LabelEncoder()
# #X[:,2] = label_encoder_2.fit_transform(values[:,2])
# X[:,2] = label_encoder_2.fit_transform(datasetpre.values[:,2])
# # print(integer_encoded)

# # Onehotencoder will make the matrix really large, not fit second-based data structure
# #onehotencoder = OneHotEncoder(categorical_features = "all")
# #X = onehotencoder.fit_transform(X).toarray()
# # ensure all data is float
# X = X.astype('float32')
# # normalize features
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(X)
# frame as supervised learning
reframed = series_to_supervised(X, N_TIME_STEPS, 1)
print(reframed.shape)

# # binary encode
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)
# # invert first example
# inverted = label_encoder.inverse_transform([argmax(onehot_encoded[1, :])])
# print(inverted)

In [ ]:
y_new = y[:-10]

In [ ]:
# split into train and test sets
X = reframed.values
N_TRAIN_TIME = 591200
train = X[:N_TRAIN_TIME, :]
test = X[N_TRAIN_TIME:, :]
# split into input and outputs
N_OBS = N_TIME_STEPS * N_FEATURES
train_X = train[:, :N_OBS]
train_y = y_new[:N_TRAIN_TIME]
test_X = test[:, :N_OBS]
test_y = y_new[N_TRAIN_TIME:]
print(train_X.shape, len(train_X), train_y.shape)
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], N_TIME_STEPS, N_FEATURES))
test_X = test_X.reshape((test_X.shape[0], N_TIME_STEPS, N_FEATURES))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
from keras import metrics
from keras import backend as K

# Below is for F1 metrics
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
#         """Recall metric.

#         Only computes a batch-wise average of recall.

#         Computes the recall, a metric for multi-label classification of
#         how many relevant items are selected.
#         """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
#         """Precision metric.

#         Only computes a batch-wise average of precision.

#         Computes the precision, a metric for multi-label classification of
#         how many selected items are relevant.
#         """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

start = time.time()

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
#model.compile(loss='mae', optimizer='adam')
#model.compile(loss='mae', optimizer='adam', metrics=[metrics.mae, metrics.categorical_accuracy])
model.compile(loss='mae',
              optimizer= "adam",
              metrics=[f1, metrics.binary_accuracy, metrics.mean_squared_error, metrics.mean_absolute_error])
# fit network
#history = model.fit(train_X, train_y, epochs=6, batch_size=1440, validation_data=(test_X, test_y), verbose=2, shuffle=False)
history = model.fit(train_X, train_y, epochs=50, batch_size=3000, 
                    validation_data=(test_X, test_y), shuffle=True)
score = model.evaluate(test_X, test_y, batch_size=3000)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

end = time.time()

time_elapse = end - start
print(time_elapse)

In [ ]:
y_pred = model.predict_classes(test_X, batch_size=10, verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(test_y, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
scores = model.evaluate(test_X, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("\n%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
print("\n%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
print("\n%s: %.2f%%" % (model.metrics_names[4], scores[4]*100))

In [ ]:
# Precision, recall, f score

from sklearn.metrics import precision_recall_fscore_support

score_spt_macro = precision_recall_fscore_support(test_y, y_pred, average='macro')
score_spt_micro = precision_recall_fscore_support(test_y, y_pred, average='micro')
score_spt_weighted = precision_recall_fscore_support(test_y, y_pred, average='weighted')
score_spt_none = precision_recall_fscore_support(test_y, y_pred, average=None)

print('Score support of precision recall and f1 score: macro: {}, micro: {}, weighted: {}, none: {}'.format(score_spt_macro, score_spt_micro, score_spt_weighted, score_spt_none))
print('Macro precision is: {}, micro precision is: {}, weighted precision is: {}'.format(score_spt_macro[0], score_spt_micro[0], score_spt_weighted[0]))
print('Macro recall is: {}, micro recall is: {}, weighted recall is: {}'.format(score_spt_macro[1], score_spt_micro[1], score_spt_weighted[1]))
print('Macro f1 score is: {}, micro f1 score is: {}, weighted f1 score is: {}'.format(score_spt_macro[2], score_spt_micro[2], score_spt_weighted[2]))
print('Score support of precision recall and f1 score: None: {}'.format(score_spt_none))

### Random Forest - 29 Features for Biweekly Prediction

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

start = time.time()

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X, y)

print ("Accuracy is %s" 
        % (accuracy_score(y_test, clf.predict(X_test))))

end = time.time()

print(end - start)

### Confusion Matrix - RAF

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
# Precision, recall, f score

from sklearn.metrics import precision_recall_fscore_support

score_spt_macro = precision_recall_fscore_support(y_test, y_pred, average='macro')
score_spt_micro = precision_recall_fscore_support(y_test, y_pred, average='micro')
score_spt_weighted = precision_recall_fscore_support(y_test, y_pred, average='weighted')
score_spt_none = precision_recall_fscore_support(y_test, y_pred, average=None)

print('Score support of precision recall and f1 score: macro: {}, micro: {}, weighted: {}, none: {}'.format(score_spt_macro, score_spt_micro, score_spt_weighted, score_spt_none))
print('Macro precision is: {}, micro precision is: {}, weighted precision is: {}'.format(score_spt_macro[0], score_spt_micro[0], score_spt_weighted[0]))
print('Macro recall is: {}, micro recall is: {}, weighted recall is: {}'.format(score_spt_macro[1], score_spt_micro[1], score_spt_weighted[1]))
print('Macro f1 score is: {}, micro f1 score is: {}, weighted f1 score is: {}'.format(score_spt_macro[2], score_spt_micro[2], score_spt_weighted[2]))
print('Score support of precision recall and f1 score: None: {}'.format(score_spt_none))

### LSTM - 5 Principal Component - for learning curve

In [ ]:
# Another test, move the ground truth one day after, use the data from previous day to predict later day
import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import time
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

num_pc = 5
processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = fileList[-1000:]

# fileList = random.sample(fileList, 500)

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

selectedList = random.sample(fileList, 2000)
restList = list(set(fileList) - set(selectedList))

scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, num_pc])
y = np.zeros([1, 1])

for f in selectedList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1 = data['w_up_down'].values
        X1 = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data
        X1.fillna(0, inplace=True)
        X1 = X1.replace(np.inf, 0)  # Replace inf with 0
    
        X1 = scaler.fit_transform(X1)

        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1)
        X1 = ipca.transform(X1)

        y0 = np.zeros([1,1])
        y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

        X = np.vstack((X, X1))
        y = np.append(y, y1)  # stack all labels

        X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y = y[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
# Prepare a test set of data

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = fileList[:2000]

fileList = random.sample(restList, 500)

# fileList = ['002169.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, num_pc])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    X1_test = X1_test.replace(np.inf, 0)  # Replace inf with 0
    try:
        X1_test = scaler.fit_transform(X1_test)
        
        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1_test)
        X1_test = ipca.transform(X1_test)
        
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
import pandas as pd
from math import sqrt
from matplotlib import pyplot
# from sklearn import preprocessing
from numpy import array
from numpy import argmax
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


N_FEATURES = 5
N_TIME_STEPS = 10

# datasetpre1 = X  # delete 'index_col=0' in bracket because of lack of index
# datasetpre = pd.get_dummies(datasetpre1,drop_first=True)  # GET DUMMMIE and drop
# X = datasetpre.values
# # integer encode, IF OUT OF INDEX ERROR, CLOSE THE DISTRIBUTION AND RELAUNCH
# # Comment label encoder because of naming error, only onehotencoder works fine now
# label_encoder_0 = LabelEncoder()
# #X[:,0] = label_encoder_0.fit_transform(values[:,0])
# X[:,0] = label_encoder_0.fit_transform(datasetpre.values[:,0])
# label_encoder_1 = LabelEncoder()
# #X[:,1] = label_encoder_1.fit_transform(values[:,1])
# X[:,1] = label_encoder_1.fit_transform(datasetpre.values[:,1])
# label_encoder_2 = LabelEncoder()
# #X[:,2] = label_encoder_2.fit_transform(values[:,2])
# X[:,2] = label_encoder_2.fit_transform(datasetpre.values[:,2])
# # print(integer_encoded)

# # Onehotencoder will make the matrix really large, not fit second-based data structure
# #onehotencoder = OneHotEncoder(categorical_features = "all")
# #X = onehotencoder.fit_transform(X).toarray()
# # ensure all data is float
# X = X.astype('float32')
# # normalize features
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(X)
# frame as supervised learning
reframed = series_to_supervised(X, N_TIME_STEPS, 1)
print(reframed.shape)

# # binary encode
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)
# # invert first example
# inverted = label_encoder.inverse_transform([argmax(onehot_encoded[1, :])])
# print(inverted)

In [ ]:
y_new = y[:-10]

In [ ]:
# split into train and test sets
X = reframed.values
N_TRAIN_TIME = 591200
train = X[:N_TRAIN_TIME, :]
test = X[N_TRAIN_TIME:, :]
# split into input and outputs
N_OBS = N_TIME_STEPS * N_FEATURES
train_X = train[:, :N_OBS]
train_y = y_new[:N_TRAIN_TIME]
test_X = test[:, :N_OBS]
test_y = y_new[N_TRAIN_TIME:]
print(train_X.shape, len(train_X), train_y.shape)
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], N_TIME_STEPS, N_FEATURES))
test_X = test_X.reshape((test_X.shape[0], N_TIME_STEPS, N_FEATURES))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
from keras import metrics
from keras import backend as K

# Below is for F1 metrics
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
#         """Recall metric.

#         Only computes a batch-wise average of recall.

#         Computes the recall, a metric for multi-label classification of
#         how many relevant items are selected.
#         """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
#         """Precision metric.

#         Only computes a batch-wise average of precision.

#         Computes the precision, a metric for multi-label classification of
#         how many selected items are relevant.
#         """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

start = time.time()

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
#model.compile(loss='mae', optimizer='adam')
#model.compile(loss='mae', optimizer='adam', metrics=[metrics.mae, metrics.categorical_accuracy])
model.compile(loss='mae',
              optimizer= "adam",
              metrics=[f1, metrics.binary_accuracy, metrics.mean_squared_error, metrics.mean_absolute_error])
# fit network
#history = model.fit(train_X, train_y, epochs=6, batch_size=1440, validation_data=(test_X, test_y), verbose=2, shuffle=False)
history = model.fit(train_X, train_y, epochs=150, batch_size=3000, 
                    validation_data=(test_X, test_y), shuffle=True)
score = model.evaluate(test_X, test_y, batch_size=3000)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

end = time.time()

time_elapse = end - start
print(time_elapse)